## Prerequsites 

In [59]:
# Prerequsite have service account..and corrresponding key
# export GOOGLE_APPLICATION_CREDENTIALS="KEY_PATH"

# pip install -r requirements.txt
# pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

In [58]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Step0: Setup BigQuery Client by Service Account key
# Reference: https://cloud.google.com/bigquery/docs/authentication/service-account-file
key_path = "../../../.google/credentials/google_credentials.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [24]:
from google.cloud.bigquery.client import Client

def run_query_bq(client:Client, query:str):
    query_job = client.query(query)
    
    # table_row_iterator = query_job.result()
    # table_df = table_row_iterator.to_dataframe()
    
    table_df = query_job.to_dataframe()
    return table_df

## Q1

**What is count for fhv vehicles data for year 2019?**  


In [16]:
# q1.1: Create external table for fhv_tripdata in 2019 files
q1_1= """
CREATE OR REPLACE EXTERNAL TABLE
  `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019` OPTIONS( format = 'PARQUET',
    uris = ['gs://dtc_data_lake_blissful-scout-339008/raw/fhv_tripdata_2019-*.parquet']);
"""
# q1.2 Query the total rows in the table
q1_2= """
SELECT COUNT(*) FROM `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`;
"""

In [27]:
run_query_bq(client, q1_1)
run_query_bq(client, q1_2) # Answer: 42084899

,f0_
0,42084899


## Q2

**How many distinct dispatching_base_num we have in fhv for 2019?**  

In [30]:
q2 = """
SELECT
  COUNT(DISTINCT dispatching_base_num)
FROM
  `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`;
"""

In [31]:
run_query_bq(client, q2) # Answer:792

,f0_
0,792


## Q3 and Q4

**Q3: Best strategy to optimise if query always filter by dropoff_datetime and order by dispatching_base_num**  

**Q4: What is the count, estimated and actual data processed for query which counts trip between 2019/01/01 and 2019/03/31 for dispatching_base_num B00987, B02060, B02279** 


In [43]:
# Create table from external table to being able to compare the estimations
q3_0 = """
    CREATE OR REPLACE TABLE
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_non_partitioned` AS
    SELECT
      *
    FROM
      `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`;
"""

# Create partitioned table by DATE(dropoff_datetime)
q3_1 = """
    CREATE OR REPLACE TABLE
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_partitioned_Q3Q4`
    PARTITION BY
      DATE(dropoff_datetime) AS
    SELECT
      *
    FROM
      `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`;
"""

# Create partitioned+clustered table by DATE(dropoff_datetime) and dispatching_base_num 
q3_2 = """
    CREATE OR REPLACE TABLE
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_partitioned_clustered_Q3Q4`
    PARTITION BY
      DATE(dropoff_datetime)
    CLUSTER BY
      dispatching_base_num AS
    SELECT
      *
    FROM
      `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`;
"""


In [44]:
# Create prerequsite tables
run_query_bq(client, q3_0)
run_query_bq(client, q3_1)
run_query_bq(client, q3_2)

""


In [45]:
# Compare Non-Partitioned vs Only Partioned
# Estimated data to be process: 643 MB (non-partitioned)
q3_2_1 = """
    SELECT
      DISTINCT(dispatching_base_num)
    FROM
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_non_partitioned`
    WHERE
      DATE(dropoff_datetime) BETWEEN '2019-06-01'
      AND '2019-06-30';
"""

# Estimated data to be process: 30.1 MB (only partitioned)
q3_2_2 = """
    SELECT
      DISTINCT(dispatching_base_num)
    FROM
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_partitioned_Q3Q4`
    WHERE
      DATE(dropoff_datetime) BETWEEN '2019-06-01'
      AND '2019-06-30';
"""

In [47]:
# run_query_bq(client, q3_2_1)
# run_query_bq(client, q3_2_2)

In [48]:
# Compare Only Partioned vs Partitioned+Clustered
# Estimated data to be process: 400.1 MB (only partitioned)
q4_1 = """
    SELECT
      COUNT(*) AS trips
    FROM
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_partitioned_Q3Q4`
    WHERE
      DATE(dropoff_datetime) BETWEEN '2019-01-01'
      AND '2019-03-31'
      AND dispatching_base_num IN ("B00987",
        "B02060",
        "B02279");
"""

# Estimated data to be process: 400.1 MB (partitioned+clustered)
# But after the run it process only 133mb
q4_2 = """
    SELECT
      COUNT(*) AS trips
    FROM
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_partitioned_clustered_Q3Q4`
    WHERE
      DATE(dropoff_datetime) BETWEEN '2019-01-01'
      AND '2019-03-31'
      AND dispatching_base_num IN ("B00987",
        "B02060",
        "B02279");
"""

In [49]:
run_query_bq(client, q4_1)
run_query_bq(client, q4_2) # Answer: 26643

,trips
0,26643


### Q3 Discover dropoff_datetime

In [33]:
# Here checking if the column is suitable for partiioning
# As maximum 4000 partitions allowed in GCP
q3_discover_dropoff_datetime_1 = """
SELECT
  COUNT(DISTINCT DATE(dropoff_datetime))
FROM
  `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`;
"""

# To see how droppoff date(dropoff_datetime) is balanced in the table
# As we are interested to have balanced partitions/clusters
q3_discover_dropoff_datetime_2 = """
SELECT
  DATE(dropoff_datetime) AS dropoff_date,
  COUNT(DATE(dropoff_datetime)) AS counter
FROM
  `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`
GROUP BY
  dropoff_date
ORDER BY
  counter DESC;
"""

In [34]:
run_query_bq(client, q3_discover_dropoff_datetime_1) # Ans:549 therefore suitable for the partitioning as it is < 4000

,f0_
0,549


In [35]:
run_query_bq(client, q3_discover_dropoff_datetime_2)

,dropoff_date,counter
0,2019-01-26,925126
1,2019-01-12,888643
2,2019-01-31,888211
3,2019-01-25,850977
4,2019-01-19,845616
...,...,...
544,2091-10-11,1
545,2020-10-26,1
546,2091-10-28,1
547,2109-05-12,1


### Q3 Discover dispatching_base_num

In [36]:
q3_discover_dispatching_base_num = """
    SELECT
      `dispatching_base_num`,
      COUNT(dispatching_base_num) AS counter
    FROM
      `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`
    GROUP BY
      dispatching_base_num
    ORDER BY
      counter DESC;
"""

In [37]:
run_query_bq(client, q3_discover_dispatching_base_num)

,dispatching_base_num,counter
0,B02510,4623412
1,B02764,1662983
2,B02765,1059883
3,B02875,1009567
4,B02800,1004974
...,...,...
787,B03242,1
788,B03245,1
789,B03247,1
790,B03131,1


## Q5

In [53]:
# Create clustered table by dispatching_base_num and SR_Flag
q5_1 = """
    CREATE OR REPLACE TABLE
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_partitioned_clustered_Q5`
    CLUSTER BY
      dispatching_base_num,
      SR_Flag AS
    SELECT
      *
    FROM
      `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`;
"""

In [54]:
run_query_bq(client, q5_1)

""


In [56]:
# Compare Non Partioned vs Clustered
# Estimated data to be process: 363 MB (non partitioned)
q5_2_1 = """
    SELECT
      COUNT(*) AS trips
    FROM
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_non_partitioned`
    WHERE
      SR_Flag=5
      AND dispatching_base_num IN ("B00987",
        "B02060",
        "B02279");
"""

# Estimated data to be process: 363 MB(partitioned+clustered)
# But after the run it process only 9,9mb
q5_2_2 = """
    SELECT
      COUNT(*) AS trips
    FROM
      `blissful-scout-339008.trips_data_all.fhv_tripdata_2019_partitioned_clustered_Q5`
    WHERE
      SR_Flag=5
      AND dispatching_base_num IN ("B00987",
        "B02060",
        "B02279");
"""

In [57]:
run_query_bq(client, q5_2_1)
run_query_bq(client, q5_2_2)

,trips
0,0


### Q5 Discover SR_FLAG

In [51]:
q5_discover_sr_flag = """
    SELECT
      `SR_Flag`,
      COUNT(SR_Flag) AS counter
    FROM
      `blissful-scout-339008.trips_data_all.external_fhv_tripdata_2019`
    GROUP BY
      SR_Flag
    ORDER BY
      counter DESC;
"""

In [52]:
run_query_bq(client, q5_discover_sr_flag)

,SR_Flag,counter
0,1.0,2369823
1,2.0,1874968
2,3.0,749388
3,4.0,229875
4,5.0,87381
5,6.0,36969
6,7.0,16437
7,8.0,7843
8,9.0,4067
9,10.0,2275


## Q6

**What improvements can be seen by partitioning and clustering for data size less than 1 GB?**


For the data less than 1GB partitioning and clustering does not add imporevements but cost as the metadata needs to processed additionally.

## Q7

**In which format does BigQuery save data**

In Columnar format